## Colab notebook for collecting ChatGPT answers from official ChatGPT API:

https://api.openai.com/v1/chat/completions.

### download raw file from: https://github.com/albert-jin/Fintech-Key-Phrase/blob/main/Fintech-Key-Phrase/dataset_final/train.json  或者使用 /train.pickle

### then place it into /content/ as /content/train.json  或者 /content/train.pickle

In [ ]:
# !curl https://github.com/albert-jin/Fintech-Key-Phrase/blob/main/Fintech-Key-Phrase/dataset_final/train.json#
# !move Fintech-Key-Phrase/blob/main/Fintech-Key-Phrase/dataset_final/train.json /content/train.json

In [1]:
# 安装实验环境依赖包
!pip3 install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.7 MB/s eta 0:00:00


## 下面是text-davinci-002的调用，效果不好舍弃，尝试利用'gpt-3.5-turbo'.

In [2]:
import openai

# 设置API密钥
api_key = "sk-HWoNMkmaO===v9QNetmR2TZT3BlbkFJNggiubRKtAiJMCW8Nyeb".replace("===","")
openai.api_key = api_key
# 设置其他请求的参数
engine = "text-davinci-002" # 'gpt-3.5-turbo'  # 使用gpt-3.5-turbo，这是目前最好的模型 # gpt-3.5-turbo  # "text-davinci-002"
max_tokens=1000
temperature=0.5
choices=1

def augment_text_chatgpt_davinci(sentence, key_phrases):
    """
    使用ChatGPT API对句子进行数据增强, 类语义扩充。

    参数:
    sentence (str): 需要数据增强的句子
    key_phrases (list of str): 预设值的key phrases

    返回: 一个扩充增强后的相似句子的列表 => (修改为ChatGPT的所有回答)
    """

    # 准备提示
    if len(key_phrases) > 1:
      prompt = f'请帮我完成以下数据增强: 给定句子: "{sentence}". \n 在保留句子原本语义同时进行改写, \n 并保留 {",".join(key_phrases[:-1])} 和 {key_phrases[-1]} 这些词汇, 要求写三个即可。'
    else:
      prompt = f'请帮我完成以下数据增强: 给定句子: "{sentence}". \n 在保留句子原本语义同时进行改写, \n 并保留 {key_phrases[0]} 这个词汇, 要求写三个即可。'
    try:
        # 向API发出请求并获取响应
        response = openai.Completion.create(
            engine=engine,
            prompt=prompt,
            max_tokens=max_tokens,
            n=choices,
            stop=None,
            temperature=temperature,
        )
    except openai.OpenAIError as e:
        print(f"Error during API request: {e}, sentence: {sentence}")
        return ''

    # 解析API响应并提取预测类标签
    try:
        predicted_category = response.choices[0].text.strip()
    except (AttributeError, KeyError) as e:
        print(f"Error in parsing API response: {e}, sentence: {sentence}")
        return ''

    # 返回ChatGPT的完整回复
    return predicted_category


# 示例用法
# sentence = "二是结合市场需求及时调整产品结构，恢复钾碱生产，加大特种树脂开拓力度"
# keyphrases = ["钾碱", "特种树脂"]

# answer_chatgpt = augment_text_chatgpt_davinci(sentence, keyphrases)
# print(f"The response of chatgpt: \n {answer_chatgpt}")

## 利用gpt-3.5-turbo结合request请求撰写脚本.

In [3]:
import json
import requests
engine = 'gpt-3.5-turbo'  # 改为 'gpt-3.5-turbo' 模型
# 为API请求准备URL、头部和负载
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + api_key
}

def augment_text_chatgpt_turbo(sentence, key_phrases):
    """ 采用 gpt-3.5-turbo 完成augment_text_chatgpt_davinci功能 """
    # 准备提示
    if len(key_phrases) > 1:
      prompt = f'请帮我完成以下数据增强: 给定句子: "{sentence}". \n 在保留句子原本语义同时进行改写, \n 并保留 {",".join(key_phrases[:-1])} 和 {key_phrases[-1]} 这些词汇, 要求写三个即可。'
    else:
      prompt = f'请帮我完成以下数据增强: 给定句子: "{sentence}". \n 在保留句子原本语义同时进行改写, \n 并保留 {key_phrases[0]} 这个词汇, 要求写三个即可。'

    messages = [{"role": "user", "content": prompt}]

    payload = json.dumps({
        "model": engine,
        "messages": messages,
        "max_tokens": max_tokens,
        "n": choices,
        "stop": None,
        "temperature": temperature,
    })

    try:
        # 向API发出请求并获取响应
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error during API request: {e}, sentence: {sentence}")
        return ''
    response_data = response.json()
    return [response_data["choices"][i]["message"]["content"].strip() for i in range(choices)]

# 示例用法
# sentence = "二是结合市场需求及时调整产品结构，恢复钾碱生产，加大特种树脂开拓力度"
# keyphrases = ["钾碱", "特种树脂"]

# answers_chatgpt = augment_text_chatgpt_turbo(sentence, keyphrases)
# print(f"The response of chatgpt: \n {answers_chatgpt[0]}")

In [ ]:
# {"text": "3、人工智能，随着末端感知、边缘计算等技术应用", "label": {"financial_entity": {"人工智能": [[2, 5]], "末端感知": [[9, 12]]}}}
## 一条train.json中的示例

In [ ]:
# -*- coding: utf-8 -*-
from tqdm import tqdm
import pickle
import os, time, random
use_pickle = True
if use_pickle:
  source_file_ChatGPT = f'/content/train.pickle'
else:
  source_file_ChatGPT = f'/content/train.json'
answers_file_ChatGPT = f'/content/answers_ChatGPT.json'  # 里面也是每一行存储一个json 字典对象，分别为四个键值：在text-> sentence, label-> keyphrases 基础上，新增了ChatGPT_answers 和 index (用作断点续下载)
if not os.path.exists(answers_file_ChatGPT):
    open(answers_file_ChatGPT, 'wt', encoding='utf-8').close()
collected_indexes = set()
with open(answers_file_ChatGPT, 'rt', encoding='utf-8') as inp:
  line = inp.readline()
  while line:
      collected_indexes.add(eval(line.strip())['index'])  # collected_indexes中提前添加 已请求过的ChatGPT增强文本
      line = inp.readline()
old_time = time.time()
with open(answers_file_ChatGPT, 'at', encoding='utf-8') as oup:
  if use_pickle:
    f = open(source_file_ChatGPT, 'rb')
    lines = pickle.load(f)
    f.close()
  else:
    inp = open(source_file_ChatGPT,'rt', encoding='utf-8')
    lines = inp.readlines()
    inp.close()
  print('示例：', lines[0]) # 展示一下
  for idx, line in tqdm(enumerate(lines)):
    if idx in collected_indexes:
      continue
    collected_indexes.add(idx)
    obj = json.loads(line.strip())
    sent, keyphrases = obj['text'], list(obj['label']['financial_entity'].keys())
    answers_chatgpt = augment_text_chatgpt_turbo(sent, keyphrases)
    print('Sentence: ', sent, '\n, ChatGPT: ', answers_chatgpt)
    json_row = {'index': idx, "sentence": sent, 'ChatGPT_answers': answers_chatgpt, "label": keyphrases}
    oup.write(json.dumps(json_row, ensure_ascii=False) + '\n')
    new_time = time.time()
    print(f'单次请求耗时: {new_time-old_time} s.')
    old_time = new_time
    oup.flush()
    time.sleep(random.random() + 2)

示例： {"text": "公司生产销售的化学原料药主要产品退热冰及阿司匹林、格列喹酮，长年来一直以其质量稳定可靠", "label": {"financial_entity": {"化学原料药": [[7, 11]], "退热冰及阿司匹林": [[16, 23]], "格列喹酮": [[25, 28]]}}}



0it [00:00, ?it/s]

Sentence:  很大程度是受热处理工艺及装备的制约 
, ChatGPT:  ['1. 受热处理工艺和装备的限制在很大程度上影响了其发展。\n2. 很大程度上受热处理工艺和装备的不足制约了其进一步发展。\n3. 要想发展，必须克服受热处理工艺和装备的制约。']
单次请求耗时: 8.121538162231445 s.


187it [00:11, 16.87it/s]

Sentence:  继续围绕“智慧城市服务与运营商”的战略定位 
, ChatGPT:  ['1. 持续聚焦“智慧城市服务与运营商”的战略定位。\n2. 在“智慧城市服务与运营商”的战略定位下不断探索创新。\n3. 以“智慧城市服务与运营商”的战略定位为引领，积极推进相关业务发展。']
单次请求耗时: 12.817652463912964 s.


...

Sentence:  一方面进一步加大OGM、G5oncell、G5CTPSensor、AR镀膜、AG膜等技术及产品的开发和市场推广力度 
, ChatGPT:  ['1. 为了进一步推广OGM、G5oncell、G5CTPSensor、AR镀膜和AG膜等技术和产品，我们将加大市场推广力度。\n2. 除了加强OGM、G5oncell、G5CTPSensor、AR镀膜和AG膜等技术和产品的开发外，我们还将加强市场推广工作。\n3. 我们将继续致力于开发和推广OGM、G5oncell、G5CTPSensor、AR镀膜和AG膜等技术和产品，以满足市场需求。']
单次请求耗时: 22.880346298217773 s.


1437it [3:33:17, 23.35s/it]

Sentence:  借助政府大力推动建筑节能的政策，加大保温板阻燃综合性能的宣传 
, ChatGPT:  ['1. 政府大力推动建筑节能政策，加强保温板阻燃综合性能的宣传，有助于提高建筑节能水平。\n2. 借助政府政策的支持，建筑节能得到了更大的推动力，同时加强保温板阻燃性能的宣传也是必要的。\n3. 为了更好地推动建筑节能，政府加大了保温板阻燃综合性能的宣传力度，以提高人们对建筑节能和保温板阻燃的认识。']
单次请求耗时: 24.718554496765137 s.


1438it [3:33:42,  8.92s/it]


KeyboardInterrupt: ignored

In [ ]:
# 继续跑上面的代码，重复执行，断点继续请求ChatGPT
# -*- coding: utf-8 -*-
from tqdm import tqdm
import pickle
import os, time, random
use_pickle = True
if use_pickle:
  source_file_ChatGPT = f'/content/train.pickle'
else:
  source_file_ChatGPT = f'/content/train.json'
answers_file_ChatGPT = f'/content/answers_ChatGPT.json'  # 里面也是每一行存储一个json 字典对象，分别为四个键值：在text-> sentence, label-> keyphrases 基础上，新增了ChatGPT_answers 和 index (用作断点续下载)
if not os.path.exists(answers_file_ChatGPT):
    open(answers_file_ChatGPT, 'wt', encoding='utf-8').close()
collected_indexes = set()
with open(answers_file_ChatGPT, 'rt', encoding='utf-8') as inp:
  line = inp.readline()
  while line:
      collected_indexes.add(eval(line.strip())['index'])  # collected_indexes中提前添加 已请求过的ChatGPT增强文本
      line = inp.readline()
old_time = time.time()
with open(answers_file_ChatGPT, 'at', encoding='utf-8') as oup:
  if use_pickle:
    f = open(source_file_ChatGPT, 'rb')
    lines = pickle.load(f)
    f.close()
  else:
    inp = open(source_file_ChatGPT,'rt', encoding='utf-8')
    lines = inp.readlines()
    inp.close()
  print('示例：', lines[0]) # 展示一下
  for idx, line in tqdm(enumerate(lines)):
    if idx in collected_indexes:
      continue
    collected_indexes.add(idx)
    obj = json.loads(line.strip())
    sent, keyphrases = obj['text'], list(obj['label']['financial_entity'].keys())
    answers_chatgpt = augment_text_chatgpt_turbo(sent, keyphrases)
    if answers_chatgpt == '':
      time.sleep(5+3*random.random())
    print('Sentence: ', sent, '\n, ChatGPT: ', answers_chatgpt)
    json_row = {'index': idx, "sentence": sent, 'ChatGPT_answers': answers_chatgpt, "label": keyphrases}
    oup.write(json.dumps(json_row, ensure_ascii=False) + '\n')
    new_time = time.time()
    print(f'单次请求耗时: {new_time-old_time} s.')
    old_time = new_time
    oup.flush()
    time.sleep(random.random() + 2)

示例： {"text": "公司生产销售的化学原料药主要产品退热冰及阿司匹林、格列喹酮，长年来一直以其质量稳定可靠", "label": {"financial_entity": {"化学原料药": [[7, 11]], "退热冰及阿司匹林": [[16, 23]], "格列喹酮": [[25, 28]]}}}



0it [00:00, ?it/s]

Sentence:  制冷压缩机产品主要应用于中央空调、地源热泵、风冷热泵、储冰系统、中低温冷藏系统等行业。空气压缩机产品主要应用于机械、电子电力、化工、冶金、医药等行业 
, ChatGPT:  ['1. 制冷压缩机产品广泛应用于中央空调、地源热泵、风冷热泵、储冰系统、中低温冷藏系统等多个领域。空气压缩机产品则主要用于机械、电子电力、化工、冶金和医药行业。\n2. 中央空调、地源热泵、风冷热泵等行业广泛使用制冷压缩机产品，而机械、电子电力、化工、冶金和医药等领域则常用空气压缩机产品。\n3. 制冷压缩机产品被广泛应用于中央空调、地源热泵、风冷热泵、储冰系统和冷藏等多个行业，而空气压缩机产品则主要用于机械、电子电力、化工、冶金和医药领域。']
单次请求耗时: 25.074750900268555 s.


1478it [00:39, 53.87it/s]

Sentence:  持续提升具有更低电阻、支持主动笔操作等性能优势的单层玻璃金属网格（OGM）结构的中大尺寸一体化电容式触摸屏产品的技术性能和成本竞争力 
, ChatGPT:  ['1. 通过优化单层玻璃金属网格（OGM）结构，进一步降低电阻，支持主动笔操作，提高中大尺寸一体化电容式触摸屏的技术性能，提高其成本竞争力。\n2. 利用单层玻璃金属网格（OGM）结构，实现更低电阻和支持主动笔操作的中大尺寸一体化电容式触摸屏产品。同时，通过技术优化，提高其性能和降低成本，增强其竞争力。\n3. 单层玻璃金属网格（OGM）结构的优化，可实现中大尺寸一体化电容式触摸屏产品的更低电阻和支持主动笔操作。进一步提升技术性能和降低成本，可增强其竞争力。']
单次请求耗时: 25.639225244522095 s.


...

1522it [15:56, 22.05s/it]

Sentence:  国内设备制造企业主要以仿制为主，研发投入较少 
, ChatGPT:  ['1. 国内的设备制造企业以仿制为主，研发方面的投入相对较少。\n2. 以仿制为主是国内设备制造企业的主要特点，而在研发方面的投入则相对较少。\n3. 尽管国内设备制造企业以仿制为主，但在研发方面的投入却相对较低。']
单次请求耗时: 14.971413135528564 s.


1523it [16:12, 20.04s/it]

Sentence:  公司与国家中药管理局中药资源中心、人参国际标准科技联盟、中科院北京基因组所、吉林省中医药管理局、长春中医药大学 
, ChatGPT:  ['1. 中药资源中心、人参国际标准科技联盟、中科院北京基因组所、吉林省中医药管理局以及长春中医药大学都是与中药和人参相关的机构。\n2. 中药管理局的人参标准与人参国际标准科技联盟的标准相似，这对中药产业的发展非常重要。\n3. 中药资源中心的人参种植技术在吉林省中医药管理局和长春中医药大学得到了广泛应用。']
单次请求耗时: 25.89157199859619 s.


1524it [16:37, 21.69s/it]